In [6]:
import threading
import time

count1 = 0
count2 = 0
count3 = 0
count4 = 0

# 添加停止标志
stop_flag = threading.Event()
mutex_count = threading.Lock()

def worker1():
    global count1
    for _ in range(6):
        if stop_flag.is_set():  # 检查停止标志
            break
        mutex_count.acquire()
        count1 += 1
        time.sleep(1)
        mutex_count.release()

def worker2():
    global count2
    for _ in range(6):
        if stop_flag.is_set():  # 检查停止标志
            break
        mutex_count.acquire()
        count2 += 1
        time.sleep(1)
        mutex_count.release()

def worker3():
    global count3
    for _ in range(6):
        if stop_flag.is_set():  # 检查停止标志
            break
        count3 += 1
        time.sleep(1)

def worker4():
    global count4
    for _ in range(6):
        if stop_flag.is_set():  # 检查停止标志
            break
        count4 += 1
        time.sleep(1)

thread1 = threading.Thread(target=worker1)
thread2 = threading.Thread(target=worker2)
thread3 = threading.Thread(target=worker3)
thread4 = threading.Thread(target=worker4)

# 设置为守护线程
thread1.daemon = True
thread2.daemon = True
thread3.daemon = True
thread4.daemon = True

thread1.start()
thread2.start()
thread3.start()
thread4.start()

time.sleep(6)  

print(f"count1: {count1}")
print(f"count2: {count2}")
print(f"count3: {count3}")
print(f"count4: {count4}")
print("所有线程已完成")

count1: 3
count2: 3
count3: 6
count4: 6
所有线程已完成


In [ ]:
import threading
import time
import random

# 银行账户类
class BankAccount:
    def __init__(self, initial_balance=1000):
        self.balance = initial_balance
        self.mutex = threading.Lock()  # 每个账户都有自己的互斥锁
        self.transaction_count = 0
    
    def transfer_to(self, target_account, amount):
        """向目标账户转账"""
        # 为了避免死锁，总是按照账户的内存地址顺序获取锁
        first_lock = self.mutex if id(self) < id(target_account) else target_account.mutex
        second_lock = target_account.mutex if id(self) < id(target_account) else self.mutex
        
        with first_lock:
            with second_lock:
                if self.balance >= amount:
                    print(f"🔄 转账开始: 从账户{id(self)%1000} -> 账户{id(target_account)%1000}, 金额: {amount}元")
                    self.balance -= amount
                    time.sleep(0.1)  # 模拟银行处理时间
                    target_account.balance += amount
                    self.transaction_count += 1
                    target_account.transaction_count += 1
                    print(f"✅ 转账成功! 余额: {self.balance}元 -> {target_account.balance}元")
                    return True
                else:
                    print(f"❌ 余额不足! 当前余额: {self.balance}元, 尝试转账: {amount}元")
                    return False

# 游戏控制器
class BankGame:
    def __init__(self):
        self.accounts = [BankAccount(1000) for _ in range(3)]  # 3个账户
        self.game_start_event = threading.Event()  # 游戏开始事件
        self.game_stop_event = threading.Event()   # 游戏结束事件
        self.players = []
        
    def create_player(self, player_id):
        """创建一个玩家线程"""
        def player_action():
            print(f"🎮 玩家{player_id} 等待游戏开始...")
            self.game_start_event.wait()  # 等待游戏开始信号
            
            while not self.game_stop_event.is_set():
                # 随机选择两个不同的账户
                from_account = random.choice(self.accounts)
                to_account = random.choice([acc for acc in self.accounts if acc != from_account])
                
                # 随机转账金额
                amount = random.randint(50, 200)
                
                # 执行转账
                from_account.transfer_to(to_account, amount)
                
                # 随机等待一段时间再进行下次转账
                time.sleep(random.uniform(0.5, 1.5))
            
            print(f"🏁 玩家{player_id} 游戏结束")
        
        return threading.Thread(target=player_action, name=f"Player-{player_id}")
    
    def start_game(self, duration=10):
        """开始游戏"""
        print("🎯 银行转账竞赛即将开始!")
        print(f"📊 初始状态:")
        for i, account in enumerate(self.accounts):
            print(f"   账户{i+1}: {account.balance}元")
        
        # 创建4个玩家
        self.players = [self.create_player(i+1) for i in range(4)]
        
        # 启动所有玩家线程
        for player in self.players:
            player.start()
        
        time.sleep(1)  # 让所有玩家准备好
        
        print(f"🚀 游戏开始! 持续{duration}秒")
        self.game_start_event.set()  # 发送游戏开始信号
        
        # 游戏持续指定时间
        time.sleep(duration)
        
        print("⏰ 时间到! 游戏结束")
        self.game_stop_event.set()  # 发送游戏结束信号
        
        # 等待所有玩家线程结束
        for player in self.players:
            player.join()
        
        self.show_results()
    
    def show_results(self):
        """显示游戏结果"""
        print("\n" + "="*50)
        print("🏆 游戏结果统计")
        print("="*50)
        
        total_balance = 0
        total_transactions = 0
        
        for i, account in enumerate(self.accounts):
            print(f"账户{i+1}: 余额 {account.balance}元, 交易次数 {account.transaction_count}")
            total_balance += account.balance
            total_transactions += account.transaction_count
        
        print(f"\n总余额: {total_balance}元 (应该是3000元)")
        print(f"总交易次数: {total_transactions//2}")  # 每笔交易计算了两次
        
        if total_balance == 3000:
            print("✅ 余额校验通过! Mutex保护有效!")
        else:
            print("❌ 余额不匹配! 可能存在并发问题!")

# 创建并运行游戏
print("🎮 欢迎来到银行转账竞赛!")
game = BankGame()
game.start_game(duration=8)

# 🏦 银行转账竞赛游戏

这是一个模拟银行转账的多线程游戏，帮助理解：
- **Mutex（互斥锁）**：保护银行账户余额不被同时修改
- **多线程**：多个客户同时进行转账操作
- **Event（事件）**：控制游戏的开始和结束

## 游戏规则：
1. 有3个银行账户，初始余额各为1000元
2. 有4个客户（线程）同时进行转账
3. 每次转账金额为50-200元随机
4. 游戏持续10秒
5. 观察有无mutex保护时的差异

## 🔍 对比实验：没有Mutex保护会发生什么？

现在让我们运行一个没有互斥锁保护的版本，看看会发生什么可怕的事情！

In [ ]:
# 危险版本：没有Mutex保护的银行账户
class UnsafeBankAccount:
    def __init__(self, initial_balance=1000):
        self.balance = initial_balance
        self.transaction_count = 0
    
    def unsafe_transfer_to(self, target_account, amount):
        """不安全的转账操作 - 没有互斥锁保护"""
        if self.balance >= amount:
            print(f"🔄 转账开始: 从账户{id(self)%1000} -> 账户{id(target_account)%1000}, 金额: {amount}元")
            
            # 这里没有锁保护，多个线程可能同时修改余额！
            old_balance = self.balance
            time.sleep(0.1)  # 模拟银行处理时间 - 这期间其他线程可能修改余额
            self.balance = old_balance - amount  # 💀 竞态条件发生的地方
            
            target_old_balance = target_account.balance
            time.sleep(0.1)
            target_account.balance = target_old_balance + amount  # 💀 竞态条件发生的地方
            
            self.transaction_count += 1
            target_account.transaction_count += 1
            print(f"✅ 转账完成! 余额: {self.balance}元 -> {target_account.balance}元")
            return True
        else:
            print(f"❌ 余额不足! 当前余额: {self.balance}元, 尝试转账: {amount}元")
            return False

# 不安全的游戏版本
class UnsafeBankGame:
    def __init__(self):
        self.accounts = [UnsafeBankAccount(1000) for _ in range(3)]
        self.game_start_event = threading.Event()
        self.game_stop_event = threading.Event()
        self.players = []
    
    def create_player(self, player_id):
        def player_action():
            print(f"💥 危险玩家{player_id} 等待游戏开始...")
            self.game_start_event.wait()
            
            while not self.game_stop_event.is_set():
                from_account = random.choice(self.accounts)
                to_account = random.choice([acc for acc in self.accounts if acc != from_account])
                amount = random.randint(50, 200)
                
                from_account.unsafe_transfer_to(to_account, amount)
                time.sleep(random.uniform(0.3, 0.8))  # 更频繁的操作增加竞态条件概率
            
            print(f"💥 危险玩家{player_id} 游戏结束")
        
        return threading.Thread(target=player_action, name=f"UnsafePlayer-{player_id}")
    
    def start_unsafe_game(self, duration=8):
        print("\n" + "💀"*20)
        print("⚠️  危险！运行没有Mutex保护的版本")
        print("💀"*20)
        print(f"📊 初始状态:")
        for i, account in enumerate(self.accounts):
            print(f"   账户{i+1}: {account.balance}元")
        
        self.players = [self.create_player(i+1) for i in range(4)]
        
        for player in self.players:
            player.start()
        
        time.sleep(1)
        print(f"💥 危险游戏开始! 持续{duration}秒")
        self.game_start_event.set()
        
        time.sleep(duration)
        
        print("⏰ 危险游戏结束")
        self.game_stop_event.set()
        
        for player in self.players:
            player.join()
        
        self.show_unsafe_results()
    
    def show_unsafe_results(self):
        print("\n" + "💀"*50)
        print("⚠️  危险游戏结果统计")
        print("💀"*50)
        
        total_balance = 0
        total_transactions = 0
        
        for i, account in enumerate(self.accounts):
            print(f"账户{i+1}: 余额 {account.balance}元, 交易次数 {account.transaction_count}")
            total_balance += account.balance
            total_transactions += account.transaction_count
        
        print(f"\n总余额: {total_balance}元 (应该是3000元)")
        print(f"总交易次数: {total_transactions//2}")
        
        if total_balance == 3000:
            print("😲 运气不错! 这次没有发生数据竞争")
        else:
            print(f"💀 灾难! 丢失了 {3000 - total_balance}元!")
            print("🔥 这就是为什么需要Mutex保护的原因!")

# 运行危险版本
print("\n🎮 现在运行危险版本，看看没有Mutex会发生什么...")
unsafe_game = UnsafeBankGame()
unsafe_game.start_unsafe_game(duration=6)

## 🎓 学习总结

通过这个银行转账竞赛游戏，你应该能够理解：

### 🔒 Mutex（互斥锁）的作用
- **保护共享资源**：防止多个线程同时修改银行账户余额
- **避免数据竞争**：确保转账操作的原子性
- **维护数据一致性**：总余额始终保持3000元

### 🧵 多线程的概念
- **并发执行**：4个客户（线程）同时进行转账操作
- **资源共享**：所有线程共享3个银行账户
- **协调工作**：通过锁机制协调线程间的操作

### 📡 Event（事件）的用途
- **同步控制**：控制所有玩家同时开始游戏
- **通信机制**：主线程向子线程发送开始/结束信号
- **优雅退出**：确保所有线程在游戏结束时能够正确停止

### 💀 没有Mutex保护的后果
- **数据丢失**：钱可能凭空消失或增加
- **竞态条件**：多个线程同时修改同一变量
- **不可预测性**：每次运行结果都可能不同

### 🏆 最佳实践
1. **总是保护共享资源**：使用mutex保护关键数据
2. **避免死锁**：按固定顺序获取多个锁
3. **使用Event进行线程通信**：比全局变量更安全
4. **测试并发程序**：多次运行验证正确性

现在你可以尝试运行上面的代码，观察安全版本和危险版本的区别！